In [12]:
import cv2
import numpy as np

# Initialize webcam
cap = cv2.VideoCapture(0)

# Load YOLO model (download these files first)
net = cv2.dnn.readNet("yolov3-helmet.cfg", "yolov3-helmet.weights")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class names
classes = ["helmet", "no-helmet"]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]

    # Detect objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process detections
    helmet_detected = False
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5 and class_id == 0:  # Class 0 is helmet
                helmet_detected = True

    # Display status
    status = "Helmet Detected" if helmet_detected else "No Helmet"
    color = (0, 255, 0) if helmet_detected else (0, 0, 255)
    cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow("Helmet Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
